In [2]:
import ray
from ray.util.multiprocessing import Pool as RayPool

import bayesfast as bf
bf.utils.parallel.set_backend(RayPool(4))

2020-10-20 15:19:42,472	INFO services.py:1164 -- View the Ray dashboard at http://127.0.0.1:8265


In [3]:
def create_model():
    import tensorflow as tf
    
    @tf.function
    def model(x):
        return -1*(x*x)
    
    def grad(self,x):
        x  = tf.convert_to_tensor(x)
        with tf.GradientTape() as tape:
            tape.watch(x)
            res = model(x)
        return tape.gradient(res, [x])
    
    return model, grad

In [10]:
@ray.remote(num_gpus=1)
class Simulator(object):
    def __init__(self):
        self.model, self.grad = create_model()
        
    def fwd(self, x):
        return self.model(self.x)
    
    def gradient(self, x):
        return self.grad(self.x)

2020-10-20 15:20:34,031	INFO (unknown file):0 -- gc.collect() freed 32 refs in 0.06173835997469723 seconds


In [5]:
## taking forever
# NetworkActor = Simulator.remote(0.1)
# result_object_ref = NetworkActor.fwd.remote()
# ray.get(result_object_ref)

In [11]:
my_sim = Simulator.remote()

bf.utils.random.set_generator(4)
den = bf.DensityLite(logp=my_sim.fwd,grad=my_sim.gradient,input_size=1)


In [12]:
sampling_params = dict(n_chain=1, n_iter=100, n_warmup=10, x_0=0.1,
                 random_generator=None, step_size=1e-6, adapt_step_size=False,
                 metric='diag', adapt_metric=False, max_change=1000.,
                 target_accept=0.6, gamma=0.05, k=0.75, t_0=10.,
                 initial_mean=None, initial_weight=1., adapt_window=60,
                 update_window=1, doubling=True)

2020-10-20 15:20:44,053	INFO (unknown file):0 -- gc.collect() freed 38 refs in 0.06531591503880918 seconds


In [13]:
foo = bf.sample(den,sample_trace=sampling_params)
foo

ValueError: failed to get finite logp and/or grad at x_0.

2020-10-20 15:20:54,083	INFO (unknown file):0 -- gc.collect() freed 138 refs in 0.07227557897567749 seconds
(pid=25284) 2020-10-20 15:20:54,112	INFO (unknown file):0 -- gc.collect() freed 160 refs in 0.11287647695280612 seconds
